<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/wikiWord2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##学習済みDoc2Vecモデル
https://yag-ays.github.io/project/pretrained_doc2vec_wikipedia/

ライブラリ
Gensim: 3.6.0

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from gensim.models import KeyedVectors
from pprint import pprint

#Gドライブ保存後の確認
model_dir = './drive/My Drive/NLP/wiki_tohoku.model'
model = KeyedVectors.load(model_dir)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [7]:
!pip install mecab-python3

     |████████████████████████████████| 15.9MB 2.9MB/s 


In [84]:
import MeCab

def search_similar_texts(p_words):
  try:
    print(model.similarity('[ポケモン]',p_words))
  except KeyError as error:
    word_cng_list.append(cut_wd.replace('[',"").replace(']',""))

words = '奥様は魔女'
inherent_words = '[' + words + ']'
word_cng_list = []

#**************************************************************************
#そのまま対義化
#**************************************************************************
asitis_cng_words = ''
try:
  for i in range(5):
    rvs_wd = model.most_similar(positive=[inherent_words,u"逆"])[i][0]
    if inherent_words.replace('[',"").replace(']',"") != rvs_wd.replace('[',"").replace(']',""):
      asitis_cng_words = model.most_similar(positive=[inherent_words,u"逆"])[i][0]
      break
    else:
      asitis_cng_words = words + "(変換なし)"
except KeyError as error:
  #辞書に登録の無い単語の場合
  search_similar_texts(inherent_words)

#**************************************************************************
#形態素分析後に対義化
#**************************************************************************
tokenizer = MeCab.Tagger("-Ochasen")
node = tokenizer.parseToNode(words)
  
while node:
  cut_wd = node.surface

  if node.feature.split(",")[0] == u"名詞":
    try:
      for i in range(5):
        rvs_wd = model.most_similar(positive=[cut_wd,u"逆"])[i][0]
        if cut_wd != rvs_wd.replace('[',"").replace(']',""):
          #◆結合
          word_cng_list.append(rvs_wd.replace('[',"").replace(']',""))
          break
    except KeyError as error:
      print("名詞キーエラー")
      #辞書に登録の無い単語の場合
      search_similar_texts(cut_wd)

  elif node.feature.split(",")[0] == u"形容詞":
    cut_wd = node.feature.split(",")[6]

    try:
      for i in range(5):
        rvs_wd = model.most_similar(positive=[cut_wd,u"逆"])[i][0]
        if cut_wd != rvs_wd.replace('[',"").replace(']',""):
          #◆結合
          word_cng_list.append(rvs_wd.replace('[',"").replace(']',""))
          break
    except KeyError as error:
      print("形容詞キーエラー")
      #辞書に登録の無い単語の場合
      search_similar_texts(cut_wd)

  elif node.feature.split(",")[0] == u"動詞":
    cu_wd = node.feature.split(",")[6]

    try:
      for i in range(5):
        rvs_wd = model.most_similar(positive=[cut_wd,u"逆"])[i][0]
        if cut_wd != rvs_wd.replace('[',"").replace(']',""):
          #◆結合
          word_cng_list.append(rvs_wd.replace('[',"").replace(']',""))
          break
    except KeyError as error:
      print("動詞キーエラー")
      #辞書に登録の無い単語の場合
      search_similar_texts(cut_wd)

  else:
    #◆結合
    word_cng_list.append(cut_wd.replace('[',"").replace(']',""))

  node = node.next

print(words)
print(asitis_cng_words)
print(word_cng_list)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


奥様は魔女

['', '', '旦那', 'は', '魔法使い', '']


In [156]:
#r'文字列'　→　正規表現。
#u'文字列'　→　日本語。Ver.2.7のとき必要。Ver.3では付けても付けなくてもよい。
try:
#  pprint(model.most_similar(positive=[u'犬',u'動物']))
#  pprint(model.most_similar(positive=[u'団体'],negative=[u"個人"],topn=2))
#  pprint(model.most_similar(positive=[u'独立'],negative=[u"依存"],topn=2))
#  pprint(model.wv.most_similar(negative=[u'世界'], topn=10))

#is-a関係の判別

  parent_wd = "動物"
  child_wd = "[北川景子]"
  isa_val_list = []
  
  for i in range(5):
    rvs_wd = model.most_similar(positive=[child_wd])[i][0]
    if child_wd != rvs_wd.replace('[',"").replace(']',""):
      isa_val_list.append(model.similarity(parent_wd,rvs_wd))

      for ii in range(5):
        rvs_wd2 = model.most_similar(positive=[rvs_wd])[ii][0]
        if rvs_wd != rvs_wd2.replace('[',"").replace(']',""):
          isa_val_list.append(model.similarity(parent_wd,rvs_wd2))

except KeyError as error:
  print(error)

ave_val = sum(isa_val_list) / len(isa_val_list)
max_val = max(isa_val_list)
min_val = min(isa_val_list)

print("平均:",ave_val)
print("最大値:",max_val)
print("最小値:",min_val)


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


平均: 0.03867989321006462
最大値: 0.09460669
最小値: -0.051938225


In [128]:
print(model.similarity('動物','[イヌ]'))


0.70940584


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
